In [ ]:
#Import Libraries

import os
from itertools import chain
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.io.json import json_normalize
import seaborn as sns
import nltk

In [ ]:
path = r"/content/tweets_ws22_v1.json"

with open(path) as f:
   
      dump = json.loads(f.read())
      org_df = pd.json_normalize(dump)
      org_df
    
final_df = pd.DataFrame() 

for i in range(0,len(dump)):
    temp_topic_df = pd.json_normalize(dump[i]['labels']['topic'])
    temp_sentiment_df = pd.json_normalize(dump[i]['labels']['sentiment'])
    temp_topic_df.rename(columns={'id':'topic_id','tweet_id':'topic_tweet_id','user_id':'topic_user_id','ground_truth':'topic_ground_truth'}, inplace = True)
    temp_sentiment_df.rename(columns={'id':'sentiment_id','tweet_id':'sentiment_tweet_id','user_id':'sentiment_user_id','ground_truth':'sentiment_ground_truth'}, inplace = True)
    df_join_topic_sent = temp_topic_df.join(temp_sentiment_df)
    final_df = final_df.append(df_join_topic_sent)
    #final_df = pd.concat([org_df,Joined_df],axis = 1)
    
    
cols = final_df.columns
org_df[cols] = final_df[cols].values


data =  org_df.drop(columns=['source_created_at','longitude','latitude','labels.topic','labels.sentiment','topic_tweet_id','sentiment_tweet_id','author_id','source','topic_user_id','topic_ground_truth','language','id','source_id','relevant','sentiment_ground_truth','sentiment_id','topic_id','sentiment_user_id'])
data

,text,topic,sentiment
0,@DSisourath The Thameslink core between London...,service,negative
1,@DulwichHistory Loving the complaint about peo...,delays,negative
2,@SW_Help .And yet you have no toilets on some ...,toilets,negative
3,@SW_Help you have no toilets on some of your t...,toilets,negative
4,@SpeedySticks007 @MrNeilJH @TLRailUK @christia...,seats,neutral
...,...,...,...
16944,Haha oh man the audio corruption on @TLRailUK ...,announcements,neutral
16945,@TLRailUK SweetIs there a plug to charge my ph...,plugs,neutral
16946,@TLRailUK now there are far fewer commuters ha...,tables,neutral
16947,@geofftech I am voting for Thameslink. 1. in a...,toilets,neutral


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
from gensim import corpora, models
import pandas as pd
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords 
import string
from gensim.models import CoherenceModel
import math
from nltk.corpus import wordnet
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = data[(data.sentiment == "negative") & (data.topic == "delays")]

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
df = data[(data.sentiment == "negative") & (data.topic == "delays")]

In [ ]:
descriptions = df['text']

In [ ]:
documents = []
texts = []
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')).union({'data', 'dataset', 'model'})
digits = set(string.digits)
for description in descriptions.iteritems():
    document_str = str(description[1]).lower()
    document_str = document_str.translate(str.maketrans('', '', string.punctuation))
    token_words = word_tokenize(document_str)
    pos_tagged = pos_tag(token_words)
    tokens = [(token, get_wordnet_pos(tag)) for token, tag in pos_tagged]
    lemma_tokens = [lemmatizer.lemmatize(token, tag) for token, tag in tokens]
    document = [w for w in lemma_tokens if (not w in stop_words) and (not w.isdigit()) ]
    documents.append(document)
    texts.append(' '.join(document))
print(documents[0])



['dulwichhistory', 'love', 'complaint', 'people', 'wait', 'minute', 'trainthey', 'clearly', 'never', 'travel', 'via', 'thameslink']


In [ ]:
dictionary = corpora.Dictionary(documents)

document_term = [dictionary.doc2bow(document) for document in documents]

In [ ]:
ldamodel = models.ldamodel.LdaModel(document_term, num_topics=20, id2word = dictionary)

In [ ]:
ldamodel.print_topics(num_words=5)

[(0,
  '0.049*"tlrailuk" + 0.033*"delay" + 0.031*"train" + 0.014*"service" + 0.010*"one"'),
 (1,
  '0.044*"tlrailuk" + 0.039*"get" + 0.032*"delay" + 0.028*"train" + 0.014*"stop"'),
 (2,
  '0.063*"due" + 0.054*"delay" + 0.034*"thameslink" + 0.033*"train" + 0.031*"minute"'),
 (3,
  '0.053*"train" + 0.052*"delay" + 0.041*"thameslink" + 0.031*"due" + 0.022*"london"'),
 (4,
  '0.055*"tlrailuk" + 0.055*"’" + 0.043*"delay" + 0.026*"day" + 0.022*"every"'),
 (5,
  '0.044*"delay" + 0.030*"tlrailuk" + 0.028*"redhill" + 0.027*"service" + 0.024*"’"'),
 (6,
  '0.054*"tlrailuk" + 0.052*"delay" + 0.022*"train" + 0.021*"min" + 0.018*"repay"'),
 (7,
  '0.045*"delay" + 0.044*"service" + 0.025*"thameslink" + 0.023*"tlrailuk" + 0.016*"cancel"'),
 (8,
  '0.054*"train" + 0.036*"delay" + 0.031*"time" + 0.029*"’" + 0.027*"tlrailuk"'),
 (9,
  '0.049*"delay" + 0.048*"due" + 0.046*"thameslink" + 0.041*"train" + 0.040*"app"'),
 (10,
  '0.045*"delay" + 0.024*"tlrailuk" + 0.021*"departure" + 0.015*"train" + 0.014*"t

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, document_term, dictionary)
vis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.108577 -0.048939       1        1  11.670725
16     0.046162  0.268586       2        1  10.767782
0     -0.103874 -0.011905       3        1   7.905311
12     0.229875 -0.090552       4        1   7.581133
6     -0.104838  0.015665       5        1   6.254204
1     -0.121485 -0.055941       6        1   5.587536
4     -0.139898  0.002069       7        1   5.570221
2      0.138654  0.027178       8        1   5.235371
19     0.216092  0.029823       9        1   4.927009
13    -0.043153 -0.030066      10        1   4.100696
11    -0.127559 -0.065305      11        1   3.784633
3      0.135212 -0.031539      12        1   3.432264
15    -0.012229 -0.074833      13        1   3.323017
18    -0.124851  0.005344      14        1   3.271032
5     -0.050917  0.075889      15        1   3.086548
9      0.198072 -0.080194      16        1   3.064772
14    -0.009893 -0.055628      17        1   2.832721
7     -0.011362  0.135928      18        1   2.802462
8     -0.021318 -0.058288      19        1   2.621434
10     0.015888  0.042710      20        1   2.181128, topic_info=           Term         Freq        Total Category  logprob  loglift
7    thameslink  2001.000000  2001.000000  Default  30.0000  30.0000
135         due  1383.000000  1383.000000  Default  29.0000  29.0000
35        train  4173.000000  4173.000000  Default  28.0000  28.0000
230           ’  1690.000000  1690.000000  Default  27.0000  27.0000
34     tlrailuk  3592.000000  3592.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
155        line    15.776744   523.265717  Topic20  -5.0096   0.3238
82       london    16.134812   964.199982  Topic20  -4.9872  -0.2650
122      expect    13.895004   348.934979  Topic20  -5.1366   0.6020
29      service    15.306897  1880.827989  Topic20  -5.0399  -0.9858
86     platform    12.439129   265.731936  Topic20  -5.2473   0.7637

[1407 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2830      6  0.853805  15mins
2830     10  0.106726  15mins
3347     17  0.964840     1hr
1131      8  0.070671     1st
1131      9  0.070671     1st
...     ...       ...     ...
2774     20  0.576957      🙄🙄
3382     16  0.900862       🤔
2677     10  0.813673       🤣
94       10  0.689766   🤦🏻‍♂️
1202     18  0.728917    🤷‍♂️

[4506 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 17, 1, 13, 7, 2, 5, 3, 20, 14, 12, 4, 16, 19, 6, 10, 15, 8, 9, 11])

In [ ]:
log_perplexity = ldamodel.log_perplexity(document_term)

串流輸出內容已截斷至最後 5000 行。
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

In [ ]:
print('Perplexity:', math.exp(log_perplexity))

Perplexity: 0.0010305886840580075


In [ ]:
coherence_model_lda = CoherenceModel(model=ldamodel, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence score:', coherence_lda)

Coherence score: 0.34529060158447644
